## 01. 라이브러리 로드

In [1]:
import pandas as pd
from pykrx import stock
from datetime import datetime, timedelta
import numpy as np
import re
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

### 02. 보통주와 우선주의 시가총액을 종목코드를 기준으로 합치는 함수 (merge_common_and_preferred_stocks 함수)
- 보통주와 우선주 구분하는 함수 (is_preferred_stock 함수)
        - 숫자 + 문자로 끝나는 경우, 숫자(5, 6, 7, 9)로 끝나는 경우 찾아서 구분
- 보통주 코드 기준으로 그룹핑 하는 함수 (get_base_code 함수)
    - 문자로 끝나는 경우 → 숫자만 추출 후 '0'으로 끝나게 변환
    - 숫자 5,6,7,9로 끝나는 경우 → '0'으로 대체
- result_df로 반환

In [ ]:
def merge_common_and_preferred_stocks(df):
    """
    보통주와 우선주의 시가총액을 종목코드를 기준으로 합치는 함수
    
    Parameters:
    df (DataFrame): 종목 데이터가 포함된 DataFrame
    
    Returns:
    DataFrame: 보통주와 우선주가 합쳐진 DataFrame
    """
    
    df_copy = df.copy()
    
    false_preferred_codes = set([
        '006800'  # 미래에셋대우우 (실제로 존재함)
    ])
    
    # 보통주/우선주 구분하는 함수
    def is_preferred_stock(code):
        if pd.isna(code):
            return False
        code_str = str(code).zfill(6)

        # 숫자 + 문자(K 등)로 끝나는 경우
        if re.match(r'^\d{5}[A-Z]$', code_str):
            return True

        # 숫자 5,6,7,9로 끝나는 경우
        if code_str[-1] in {'5', '6', '7', '9'}:
            return True

        return False
    
    # base_code 정의 (보통주 코드 기준으로 그룹핑)
    def get_base_code(code):
        if pd.isna(code):
            return ''
        code_str = str(code).zfill(6)
        
        # 문자로 끝나는 경우 → 숫자만 추출 후 '0'으로 끝나게
        if re.match(r'^\d{5}[A-Z]$', code_str):
            return code_str[:5] + '0'
        
        # 숫자 5,6,7,9로 끝나는 경우 → '0'으로 대체
        if code_str[-1] in {'5', '6', '7', '9'}:
            return code_str[:5] + '0'
        
        # 그 외는 보통주로 간주
        return code_str
    
    df_copy['종목코드'] = df_copy['종목코드'].astype(str).str.zfill(6)
    df_copy['is_preferred'] = df_copy['종목코드'].apply(is_preferred_stock)
    df_copy['base_code'] = df_copy['종목코드'].apply(get_base_code)
    
    merged_data = []
    grouped = df_copy.groupby('base_code')
    
    for base_code, group in grouped:
        common_stock = group[~group['is_preferred']]
        preferred_stocks = group[group['is_preferred']]
        
        if len(common_stock) == 1:
            base_row = common_stock.iloc[0].to_dict()
            
            base_row['시가총액'] += preferred_stocks['시가총액'].sum()
            base_row['상장주식수'] += preferred_stocks['상장주식수'].sum()
            base_row['거래량'] += preferred_stocks['거래량'].sum()
            base_row['거래대금'] += preferred_stocks['거래대금'].sum()
            base_row['우선주_포함'] = True
            base_row['우선주_수'] = len(preferred_stocks)
            base_row['우선주_종목명'] = ', '.join(preferred_stocks['종목명'].tolist())
            merged_data.append(base_row)
        else:
            for _, row in group.iterrows():
                row_dict = row.to_dict()
                row_dict['우선주_포함'] = False
                row_dict['우선주_수'] = 0
                row_dict['우선주_종목명'] = ''
                merged_data.append(row_dict)
    
    result_df = pd.DataFrame(merged_data)
    result_df = result_df.sort_values('시가총액', ascending=False).reset_index(drop=True)
    
    return result_df

### 03. 여러 연도의 시가총액 데이터를 처리하고 하나로 결합하는 함수 (process_all_years_data 함수)
- 여러 연도의 시가총액 데이터 불러오기
    - 연도 컬럼 추가
    - 보통주와 우선주 합치기 (merge_common_and_preferred_stocks 함수 이용) -> 전체 데이터에 추가

- 모든 연도 데이터 결합 및 결과 저장

In [ ]:
def process_all_years_data(start_year=2011, end_year=2023):
    """
    여러 연도의 시가총액 데이터를 처리하고 하나로 결합하는 함수
    
    Parameters:
    start_year (int): 시작 연도
    end_year (int): 종료 연도
    
    Returns:
    DataFrame: 모든 연도의 데이터가 결합된 DataFrame
    """
    
    all_data = []
    processing_summary = []
    
    for year in range(start_year, end_year + 1):
        file_name = f"data/krx/{year}_cap.xlsx"
        
        if not os.path.exists(file_name):
            print(f"{file_name} 파일을 찾을 수 없습니다. 건너뜁니다.")
            continue
        
        try:
            print(f"\n{year}년 데이터 처리 중...")
            
            df = pd.read_excel(file_name)
            original_count = len(df)
            original_market_cap = df['시가총액'].sum()
            
            df['연도'] = year
            
            # 보통주와 우선주 합치기
            merged_df = merge_common_and_preferred_stocks(df)
                   
            # 전체 데이터에 추가
            all_data.append(merged_df)
            
        except Exception as e:
            print(f"{year}년 데이터 처리 중 오류 발생: {str(e)}")
            continue
    
    # 모든 연도 데이터 결합
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        
        combined_df = combined_df.sort_values(['연도', '종목코드']).reset_index(drop=True)
        
        output_file = 'data/krx/all_years_merged_cap.xlsx'
        combined_df.to_excel(output_file, index=False)
        print(f"\n 결과가 '{output_file}' 파일로 저장되었습니다.")

        return combined_df
    else:
        return None

### 04. 메인 실행 코드

In [ ]:
if __name__ == "__main__":
    # 2013년부터 2023년까지 모든 데이터 처리
    combined_data = process_all_years_data(2011, 2023)

### 05. 시가총액 데이터 저장
- 컬럼명 재정의
- 필요한 컬럼만 뽑은 후 데이터 저장

In [ ]:
df = pd.read_excel('data/krx/all_years_merged_cap.xlsx', dtype={'종목코드' : str})
df['회계년도'] = df['연도'].astype(str) + '/12'
df['회사명'] = df['종목명']
df['거래소코드'] = df['종목코드']
df_filter = df[['회사명','거래소코드','회계년도','시가총액']]

df_filter.to_excel('dataset/시가총액(우선주포함).xlsx', index=False)